In [1]:
import xgboost as xgb
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score

In [2]:
data = pd.read_csv('OV2.csv', on_bad_lines='skip', sep=';')
data = pd.DataFrame(data)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10864 entries, 0 to 10863
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Date              10864 non-null  object 
 1   Streams           10864 non-null  int64  
 2   Viewers           10864 non-null  int64  
 3   Dominant_streams  10864 non-null  int64  
 4   Dominant_ratio    10864 non-null  float64
 5   Tags              10864 non-null  object 
 6   Name              10864 non-null  object 
 7   Title             10864 non-null  object 
 8   Beginning         10864 non-null  object 
 9   Unnamed: 9        53 non-null     object 
 10  Hour              10864 non-null  int64  
 11  Minute            10864 non-null  int64  
 12  Weekday           10864 non-null  int64  
 13  Moving_streams    10855 non-null  object 
dtypes: float64(1), int64(6), object(7)
memory usage: 1.2+ MB


In [3]:
X = data[['Hour','Minute','Weekday']]
#y = data[['Streams','Dominant_streams','Viewers', 'Dominant_ratio']]

y = data[['Moving_streams']]
#y['Moving_viewers'] = y['Moving_viewers'].str.replace(',', '.').astype(float)
y['Moving_streams'] = y['Moving_streams'].str.replace(',', '.').astype(float)
y = y.drop(y.index[0:10])
X = X.drop(X.index[0:10])

C:\Users\tymot\AppData\Local\Temp\ipykernel_2980\2667274269.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y['Moving_streams'] = y['Moving_streams'].str.replace(',', '.').astype(float)


In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.01, shuffle=False)
y_test = y_test.reset_index()

In [5]:

xgb_model = xgb.XGBRegressor(n_estimators = 1000)

#xgb_model.fit(X, y, eval_set=[(X, y), (X, y)],verbose=100)
xgb_model.fit(X_train, y_train)

y_pred = xgb_model.predict(X_test)

y_pred = y_pred.astype(int)
y_test['Moving_streams'] = y_test['Moving_streams'].astype(int)
#print(y_pred)
#print(y_test['Moving_streams'])


wynik = 0
wynik_list =[]
#For bigger games
# for i in range(len(y_test)):
#     temp = max(y_pred[i],y_test['Moving_streams'][i]) / min(y_pred[i],y_test['Moving_streams'][i])
#     temp = ((temp * 100) - 100) * 10
#     temp = 100 - temp
#     if temp > 0:
#         wynik += temp/100
#         wynik_list.append(temp)
#     else:
#         wynik += 0
#         wynik_list.append(0)

for i in range(len(y_test)):
    if y_test['Moving_streams'][i] == y_pred[i]:
        wynik += 1
        wynik_list.append(100)
    elif y_pred[i] == y_test['Moving_streams'][i] + 1 or  y_pred[i] == y_test['Moving_streams'][i] - 1:
        wynik += 0.5
        wynik_list.append(50)
    else:
        wynik += 0 
        wynik_list.append(0)

# For bigger games
# df = pd.DataFrame(y_pred)
# df['should'] = y_test['Moving_streams']
# df['result'] = wynik_list
# df.columns = ['is','should','result']
# display(df)

print(wynik/len(y_test))

df = pd.DataFrame(y_pred)
df['should'] = y_test['Moving_streams']
df['result'] = wynik_list
df.columns = ['is','should','result']
display(df)



0.28440366972477066


,is,should,result
0,4,5,50
1,4,5,50
2,4,5,50
3,5,5,100
4,5,5,100
...,...,...,...
104,12,10,0
105,12,10,0
106,11,11,100
107,11,11,100


In [6]:

new_data = pd.DataFrame({'Hour': [17], 'Minute': [0], 'Weekday': [4]})
prediction = xgb_model.predict(new_data)

print('Score: ', prediction)

Score:  [4.302878]


In [7]:
xgb_model.feature_importances_

array([0.8232592 , 0.02442234, 0.15231848], dtype=float32)

In [9]:
from sklearn.ensemble import IsolationForest

X = data[['Hour','Minute','Weekday','Streams']]

isof = IsolationForest(n_estimators=500)
isof.fit(X)

new_data = pd.DataFrame({'Hour': [9], 'Minute': [12], 'Weekday': [2], 'Streams': [5]})
prediction = isof.predict(new_data)

print('Wynik: ', prediction)


Wynik:  [1]
